In [51]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=512,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [52]:
from langchain_community.document_loaders import WikipediaLoader

docs = WikipediaLoader(query="San Francisco").load_and_split(text_splitter = text_splitter )

In [53]:
# List to hold the content of each document
doc_contents = [doc.page_content for doc in docs]

# List to hold the IDs for each document
doc_ids = list(range(1, len(docs) + 1))

In [54]:
doc_contents[0]

'San Francisco, officially the City and County of San Francisco, is a commercial, financial, and cultural center in Northern California.  With a population of 808,437 residents as of 2022, San Francisco is the fourth most populous city in the U.S. state of California. The city covers a land area of 46.9 square miles (121 square kilometers) at the end of the San Francisco Peninsula, making it the second-most densely populated large U.S. city after New York City and the fifth-most densely populated U.S.'

In [55]:
from qdrant_client import QdrantClient

# Initialize the client
client = QdrantClient(":memory:")

client.add(
    collection_name="leo_collection",
    documents=doc_contents,
    ids=doc_ids,
)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [81]:
from dspy.retrieve.qdrant_rm import QdrantRM
import dspy

qdrant_retriever_model = QdrantRM("leo_collection", client, k=10)

# # Import the Ollama model
# ollama_model = dspy.OllamaLocal(model="llama3",model_type='text',
#                                 max_tokens=350,
#                                 temperature=0.1,
#                                 top_p=0.8, frequency_penalty=1.17, top_k=40)

# dspy.settings.configure(lm= ollama_model, rm=qdrant_retriever_model)
# # end import Ollama model

# Set up OpenAI API client
import openai

openai.api_key = 'sk-proj-qAoqMn0r7W8psRGbkoFPT3BlbkFJSn8KlNb9RvHX0l2tZJLZ'

openai_model = dspy.OpenAI(model="gpt-3.5-turbo", api_key=openai.api_key)
dspy.settings.configure(lm=openai_model, rm=qdrant_retriever_model)
# end import OpenAI model

In [73]:
import dspy
def get_top_passages(question):
    retrieve = dspy.Retrieve(k=10)
    topK_passages = retrieve(question,k=10).passages
    print(f"Top {retrieve.k} passages for question: {question} \n", '-' * 30, '\n')
    for idx, passage in enumerate(topK_passages):
        print(f'{idx+1}]', passage, '\n')

In [74]:
get_top_passages("Where is San Fransisco's airport?")

Top 10 passages for question: Where is San Fransisco's airport? 
 ------------------------------ 

1] San Francisco International Airport (IATA: SFO, ICAO: KSFO, FAA LID: SFO) is the primary international airport serving the San Francisco Bay Area in the U.S. state of California. It is located in San Mateo County. 

2] SFO is the largest airport in the San Francisco Bay Area and the second-busiest in California, after Los Angeles International Airport (LAX). In 2023, it was the fourteenth-busiest airport in the United States and the 30th-busiest in the world by passenger count. It is the fifth-largest hub for United Airlines, functioning as the airline's primary transpacific gateway, and as a major maintenance hub. It also serves as a hub for Alaska Airlines. 

3] During World War II, the airport was used as a Coast Guard base and Army Air Corps training and staging base. The base was called Naval Auxiliary Air Facility Mills Field and Coast Guard Air Station, San Francisco.  Pan Ameri

In [75]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField()

In [76]:
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

In [77]:
from dsp.utils import deduplicate

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question):
        context = []

        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

In [82]:
# Ask any question you like to this simple RAG program.
my_question = "How many types of rail public transportation means in San Fransisco?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
uncompiled_baleen = SimplifiedBaleen()  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Question: How many types of rail public transportation means in San Fransisco?
Predicted Answer: There are three types of rail public transportation means in San Francisco.
Retrieved Contexts (truncated): ['== Geology ==\n\nSan Francisco Bay is thought to repres...', '== Alamo Square ==...', '== Campuses ==\n\nThe University of San Francisco offers more than 65 undergraduate and 60 graduate programs across its campuses.\nThe university\'s 55-acre main campus is known as "the Hilltop." It\'s div...']


In [83]:
openai_model.inspect_history(n=3)




Write a simple search query that will help answer a complex question.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the query}. We ...

Query: ${query}

---

Context: N/A

Question: How many types of rail public transportation means in San Fransisco?

Reasoning: Let's think step by step in order to produce the query. We know that different types of rail transportation exist, such as subways, trams, and light rail. By searching for the types of rail transportation in San Francisco, we can find the answer to our question.

Query: "types of rail public transportation in San Francisco"





Write a simple search query that will help answer a complex question.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the query}. We ...

Query: ${query}

---

Context:
[1] «== Geology ==



'\n\n\nWrite a simple search query that will help answer a complex question.\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the query}. We ...\n\nQuery: ${query}\n\n---\n\nContext: N/A\n\nQuestion: How many types of rail public transportation means in San Fransisco?\n\nReasoning: Let\'s think step by step in order to\x1b produce the query. We know that different types of rail transportation exist, such as subways, trams, and light rail. By searching for the types of rail transportation in San Francisco, we can find the answer to our question.\n\nQuery: "types of rail public transportation in San Francisco"\x1b\n\n\n\n\n\nWrite a simple search query that will help answer a complex question.\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the query}. We ...